In [2]:
import cv2
import mediapipe as mp
import numpy as np
import os
import time
import tkinter as tk
from tkinter import messagebox, filedialog
from threading import Thread

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

Mediapipe Pose Extraction

In [24]:
video_path = r'D:\MAPUA\Thesis\Video_Sample1\Video1.mp4'
output_folder = r'D:\MAPUA\Thesis\OutputVideo_Sample1'

def extract_landmarks_from_video(video_path):
    # Initialize MediaPipe Pose
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

    # Read the video
    cap = cv2.VideoCapture(video_path)

    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Create an output folder with the same name as the video file
    output_folder_path = os.path.join(output_folder, os.path.splitext(os.path.basename(video_path))[0])
    os.makedirs(output_folder_path, exist_ok=True)

    # Iterate over each frame
    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        # Convert BGR frame to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process frame with MediaPipe Pose
        results = pose.process(frame_rgb)

        # Draw landmarks on the frame
        if results.pose_landmarks:
            height, width, _ = frame.shape
            for landmark in results.pose_landmarks.landmark:
                x, y = int(landmark.x * width), int(landmark.y * height)
                cv2.circle(frame, (x, y), radius=5, color=(0, 255, 0), thickness=5)

        # Save the frame with landmarks as an image
        frame_path = os.path.join(output_folder_path, f"frame_{frame_count}.jpg")
        cv2.imwrite(frame_path, frame)

        # Display the frame
        cv2.imshow('Frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        frame_count += 1

    # Release resources
    cap.release()
    cv2.destroyAllWindows()

    print(f"Images saved to {output_folder} folder.")

extract_landmarks_from_video(video_path)

Images saved to D:\MAPUA\Thesis\OutputVideo_Sample1 folder.



Lucas-Kanade Optical Flow


In [25]:
def apply_lucas_kanade(output_folder):
    # Create the Lucas-Kanade output folder
    lk_output_folder = os.path.join("D:\MAPUA\Thesis\OutputVideo_LK", f"LK_{os.path.basename(output_folder)}")
    os.makedirs(lk_output_folder, exist_ok=True)

    # Get the list of image files in the output folder
    image_files = sorted([file for file in os.listdir(output_folder) if file.endswith('.jpg')])

    # Initialize Lucas-Kanade parameters
    lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

    prev_frame = None
    prev_points = None
    frame_count = 0

    for image_file in image_files:
        # Read the image
        image_path = os.path.join(output_folder, image_file)
        frame = cv2.imread(image_path)
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if prev_frame is None:
            prev_frame = gray_frame
            prev_points = cv2.goodFeaturesToTrack(prev_frame, maxCorners=500, qualityLevel=0.3, minDistance=7)
            continue

        # Calculate optical flow using Lucas-Kanade
        next_points, status, _ = cv2.calcOpticalFlowPyrLK(prev_frame, gray_frame, prev_points, None, **lk_params)

        # Filter points based on status
        next_points_filtered = next_points[status == 1]
        prev_points_filtered = prev_points[status == 1]

        # Draw optical flow lines on the frame
        for (prev_x, prev_y), (next_x, next_y) in zip(prev_points_filtered, next_points_filtered):
            prev_pt = (int(prev_x), int(prev_y))  # Convert to integers
            next_pt = (int(next_x), int(next_y))  # Convert to integers
            cv2.arrowedLine(frame, prev_pt, next_pt, (0, 255, 0), 2)
            
        # Save the frame with optical flow as an image
        lk_frame_path = os.path.join(lk_output_folder, f"frame_{frame_count}.jpg")
        cv2.imwrite(lk_frame_path, frame)

        # Update the previous frame and points
        prev_frame = gray_frame
        prev_points = next_points

        frame_count += 1

    print(f"Images with Lucas-Kanade optical flow saved to {lk_output_folder} folder.")


# Example usage:
output_folder = r'D:\MAPUA\Thesis\OutputVideo_Sample1\Video1'
apply_lucas_kanade(output_folder)

Images with Lucas-Kanade optical flow saved to D:\MAPUA\Thesis\OutputVideo_LK\LK_Video1 folder.


TKINTER

In [3]:
# Specify the video save directory
video_save_directory = r'D:\MAPUA\Thesis\Video_Sample1'
output_folder = r'D:\MAPUA\Thesis\OutputVideo_Sample1'

# Create the main GUI window
window = tk.Tk()
window.title("Video Recording")
window.geometry("400x200")

# Variable to store the video file name
video_file_name = ""

def start_recording():
    global video_file_name
    video_file_name = "Video_" + str(len(os.listdir(video_save_directory)) + 1) + ".mp4"

    # Prompt the user to select a video size
    video_sizes = {
        "640x480": (640, 480),
        "1280x720": (1280, 720),
        "1920x1080": (1920, 1080)
    }
    selected_size = tk.StringVar()
    size_selection_window = tk.Toplevel(window)
    size_selection_window.title("Select Video Size")
    size_selection_window.geometry("200x150")
    size_selection_label = tk.Label(size_selection_window, text="Select Video Size:")
    size_selection_label.pack(pady=10)
    size_selection = tk.OptionMenu(size_selection_window, selected_size, *video_sizes.keys())
    size_selection.pack(pady=10)

    def start_recording_with_size():
        size_selection_window.destroy()

        # Get the selected video size
        width, height = video_sizes[selected_size.get()]

        # Start recording the video using the camera
        video_path = os.path.join(video_save_directory, video_file_name)
        cap = cv2.VideoCapture(0)
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        out = cv2.VideoWriter(video_path, fourcc, 20.0, (width, height))

        start_time = time.time()
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            cv2.imshow("Recording", frame)
            out.write(frame)

            elapsed_time = time.time() - start_time
            if elapsed_time >= 10:
                break

            if cv2.waitKey(1) == ord('q'):
                break

        # Release resources
        cap.release()
        out.release()
        cv2.destroyAllWindows()

        # Display a message box after recording is complete
        messagebox.showinfo("Recording Complete", "Video recording has been saved.")

    start_button.config(state=tk.DISABLED)
    size_selection_window.protocol("WM_DELETE_WINDOW", start_button.config(state=tk.NORMAL))
    size_selection_window.protocol("WM_DELETE_WINDOW", on_closing)
    size_selection_button = tk.Button(size_selection_window, text="Start Recording", command=start_recording_with_size)
    size_selection_button.pack(pady=10)

def show_recordings():
    os.startfile(video_save_directory)

def on_closing():
    # Delete the recorded video file if the GUI window is closed
    global video_file_name
    if video_file_name:
        video_path = os.path.join(video_save_directory, video_file_name)
        if os.path.exists(video_path):
            os.remove(video_path)
    window.destroy()

# Create the "Start Recording" button
start_button = tk.Button(window, text="Start Recording", command=start_recording)
start_button.pack(pady=10)

# Create the "Show Recordings" button
show_button = tk.Button(window, text="Show Recordings", command=show_recordings)
show_button.pack(pady=10)

# Set the event handler for closing the GUI window
window.protocol("WM_DELETE_WINDOW", on_closing)

# Start the GUI event loop
window.mainloop()

KeyboardInterrupt: 

: 